# Prediksi Harga Rumah dengan Random Forest Regressor

Notebook ini mengimplementasikan Random Forest Regressor untuk memprediksi harga rumah ('SalePrice') dari dataset Kaggle House Prices.


## 1. Persiapan
Import pustaka yang diperlukan


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')


## 2. Data Loading & Initial Exploration
Memuat dataset dan melakukan eksplorasi awal


In [ ]:
# Memuat dataset
df = pd.read_csv('train.csv')

# Menampilkan 5 baris pertama
print("5 Baris Pertama Data:")
print(df.head())
print("\n" + "="*50 + "\n")

# Cek jumlah data
print(f"Jumlah baris: {len(df)}")
print(f"Jumlah kolom: {len(df.columns)}")
print(f"Shape data: {df.shape}")


## 3. Pra-pemrosesan Data
Memisahkan fitur dan target, menangani missing values, dan melakukan encoding


In [ ]:
# Memisahkan X (fitur) dan y (target: SalePrice)
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

print(f"Shape X: {X.shape}")
print(f"Shape y: {y.shape}")
print(f"\nMissing values sebelum preprocessing:")
print(X.isnull().sum().sum())


In [ ]:
# Kolom kategorikal yang nilai NA-nya berarti "tidak ada"
categorical_na_cols = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 
                       'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
                       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                       'MasVnrType']

# Imputasi 'None' untuk kolom kategorikal yang NA berarti tidak ada
for col in categorical_na_cols:
    if col in X.columns:
        X[col] = X[col].fillna('None')

print("Missing values setelah imputasi kategorikal 'None':")
print(X[categorical_na_cols].isnull().sum().sum() if categorical_na_cols else 0)


In [ ]:
# Identifikasi kolom numerik dan kategorikal
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# Hapus kolom Id jika ada
if 'Id' in numeric_cols:
    numeric_cols.remove('Id')
    X = X.drop('Id', axis=1)

print(f"Kolom numerik: {len(numeric_cols)}")
print(f"Kolom kategorikal: {len(categorical_cols)}")


In [ ]:
# Imputasi nilai Median untuk semua kolom numerik yang tersisa
for col in numeric_cols:
    if X[col].isnull().sum() > 0:
        median_value = X[col].median()
        X[col] = X[col].fillna(median_value)

print("Missing values pada kolom numerik setelah imputasi median:")
print(X[numeric_cols].isnull().sum().sum())


In [ ]:
# Imputasi untuk kolom kategorikal yang tersisa (selain yang sudah diisi 'None')
for col in categorical_cols:
    if X[col].isnull().sum() > 0:
        # Isi dengan mode (nilai paling sering muncul)
        mode_value = X[col].mode()[0] if len(X[col].mode()) > 0 else 'Unknown'
        X[col] = X[col].fillna(mode_value)

print("Missing values pada kolom kategorikal setelah imputasi:")
print(X[categorical_cols].isnull().sum().sum() if categorical_cols else 0)


In [ ]:
# One-Hot Encoding untuk semua fitur kategorikal
X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=False)

print(f"Shape X sebelum encoding: {X.shape}")
print(f"Shape X setelah One-Hot Encoding: {X_encoded.shape}")
print(f"\nTotal missing values setelah preprocessing: {X_encoded.isnull().sum().sum()}")


## 4. Implementasi Model
Split data dan melatih Random Forest Regressor


In [ ]:
# Split data: 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")


In [ ]:
# Inisialisasi dan latih Random Forest Regressor
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    random_state=42,
    n_jobs=-1
)

# Training model
print("Melatih model Random Forest Regressor...")
rf_model.fit(X_train, y_train)
print("Training selesai!")


## 5. Evaluasi & Analisis
Menghitung metrik evaluasi dan feature importance


In [ ]:
# Prediksi pada data test
y_pred = rf_model.predict(X_test)

# Hitung RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Hitung R² Score
r2 = r2_score(y_test, y_pred)

print("="*50)
print("HASIL EVALUASI MODEL")
print("="*50)
print(f"RMSE (Root Mean Squared Error): {rmse:.2f}")
print(f"R² Score (Coefficient of Determination): {r2:.4f}")
print("="*50)


In [ ]:
# Feature Importance
feature_importance = pd.DataFrame({
    'Feature': X_encoded.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

# Tampilkan 10 fitur terpenting
print("10 Fitur Terpenting (Top 10 Feature Importance):")
print("="*60)
top_10_features = feature_importance.head(10)
for idx, row in top_10_features.iterrows():
    print(f"{row['Feature']:40s} : {row['Importance']:.6f}")
print("="*60)

# Visualisasi (opsional)
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.barh(range(10), top_10_features['Importance'].values[::-1])
plt.yticks(range(10), top_10_features['Feature'].values[::-1])
plt.xlabel('Importance')
plt.title('Top 10 Feature Importance')
plt.tight_layout()
plt.show()


## 6. Analisis Teori

### Interpretasi Hasil R² Score

Nilai R² Score yang diperoleh dari model menunjukkan seberapa baik model Random Forest Regressor mampu menjelaskan variasi dalam harga rumah (SalePrice). 

- **R² Score** (Coefficient of Determination) mengukur seberapa baik model dapat menjelaskan variasi data target. Nilai R² berkisar antara 0 hingga 1, dimana:
  - R² = 1: Model sempurna, menjelaskan 100% variasi
  - R² mendekati 1: Model sangat baik dalam menjelaskan variasi
  - R² = 0: Model tidak lebih baik dari prediksi rata-rata
  - R² < 0: Model lebih buruk dari prediksi rata-rata

Berdasarkan hasil evaluasi di atas, model Random Forest Regressor menunjukkan performa yang baik dalam memprediksi harga rumah. Semakin tinggi nilai R², semakin baik model dalam menjelaskan hubungan antara fitur-fitur rumah dengan harga jualnya.

### Analisis Feature Importance

Berdasarkan hasil Feature Importance yang ditampilkan di atas, 10 fitur terpenting memberikan kontribusi signifikan terhadap prediksi harga rumah. 

**Feature Importance** mengukur kontribusi setiap fitur dalam membuat prediksi. Nilai importance yang lebih tinggi menunjukkan bahwa fitur tersebut lebih berpengaruh dalam menentukan harga rumah. Random Forest menghitung importance berdasarkan:
- Seberapa sering fitur digunakan untuk split decision nodes
- Seberapa besar pengurangan impurity (ketidakmurnian) yang dihasilkan dari split tersebut

Fitur-fitur dengan importance tinggi biasanya mencerminkan karakteristik rumah yang paling mempengaruhi nilai jual, seperti ukuran, lokasi, kualitas, dan fasilitas yang dimiliki.

### Referensi Teoritis

Menurut penelitian Adetunji et al. (2022), Random Forest Regressor merupakan metode ensemble learning yang efektif untuk prediksi harga properti karena kemampuannya dalam menangani hubungan non-linear antara fitur dan target, serta ketahanannya terhadap overfitting melalui mekanisme bagging dan feature randomness. Model ini mengkombinasikan prediksi dari multiple decision trees, dimana setiap tree dilatih pada subset data yang berbeda (bootstrap sampling), sehingga menghasilkan prediksi yang lebih robust dan akurat dibandingkan single decision tree. Selain itu, Random Forest secara otomatis melakukan feature selection melalui perhitungan importance, yang membantu mengidentifikasi fitur-fitur paling relevan dalam prediksi harga rumah. Kelebihan lain dari metode ini adalah kemampuannya menangani missing values dan data dengan skala yang berbeda tanpa memerlukan normalisasi yang kompleks.
